In [1]:
import pandas as pd
import numpy as np
import matplotlib as plt
from wordcloud import WordCloud
import re
import nltk

In [2]:
df = pd.read_csv('comments.csv');df

,event
0,il souhaite faire une loa sur une duree de 24 ...
1,il souhaite financer le vehicule au comptant o...
2,elle est a l'ecoute des offres en loa avec un ...
3,elle ne sait pas par quelle mode de financemen...
4,il souhaite un credit pour professionnel.
...,...
14054,il souhaite un credit aupres de sa banque.
14055,elle fait un credit partiel aupres de sa banque.
14056,il est a l'ecoute des offres de credit partiel.
14057,il souhaite une offre pour un credit a titre c...


# Etape 1: 
Analyse exploratoire

In [3]:
# describe
df.describe()

,event
count,14059
unique,4711
top,il souhaite un credit.
freq,375


In [4]:
df_unique = df.value_counts().head(10)
df_unique

event                                          
il souhaite un credit.                             375
il souhaite un financement en credit.              262
il souhaite un achat comptant.                     250
il est interesse par les offres de financement.    240
il est interesse par les offres de credit.         216
il est a l'ecoute des offres de financement.       213
il est interesse par les offres.                   201
il n'est pas interesse.                            194
il regle au comptant.                              183
il souhaite faire un paiement comptant.            163
dtype: int64

In [5]:
# nombre de mots
# df["nmb_words"] = df["event"].str.split(' ').str.len()
df["nmb_words"] = df["event"].apply(lambda x: len(str(x).split(' ')))

In [6]:
# nombre de charactères
df["nmb_char"] = df["event"].apply(lambda x: len(str(x)))

In [7]:
# Nombre de phrases
df["nmb_sentences"] =df["event"].apply(lambda x: len(str(x).split('.')))

In [8]:
df

,event,nmb_words,nmb_char,nmb_sentences
0,il souhaite faire une loa sur une duree de 24 ...,11,50,1
1,il souhaite financer le vehicule au comptant o...,12,68,2
2,elle est a l'ecoute des offres en loa avec un ...,13,65,3
3,elle ne sait pas par quelle mode de financemen...,10,54,2
4,il souhaite un credit pour professionnel.,6,41,2
...,...,...,...,...
14054,il souhaite un credit aupres de sa banque.,8,42,2
14055,elle fait un credit partiel aupres de sa banque.,9,48,2
14056,il est a l'ecoute des offres de credit partiel.,9,47,2
14057,il souhaite une offre pour un credit a titre c...,10,56,2


In [9]:
df_avg_words = df["nmb_words"].mean(); df_avg_words


7.852763354434882

In [10]:
df_avg_sentences = df["nmb_sentences"].mean(); df_avg_sentences

2.0767479906109965

In [11]:
df_avg_char = df["nmb_char"].mean(); df_avg_char

44.471584038694076

In [12]:
# nombre moyen de char par phrase pour chaque ligne
df["avg_char_sentences"] = df["event"].apply(lambda x: len(str(x).split('.')))
df["avg_char_sentences"] = df["nmb_char"] / df["nmb_sentences"]

In [13]:
# nombre moyen de mot par phrase pour chaque ligne
df["avg_words_sentences"] = df["event"].apply(lambda x: len(str(x).split('.')))
df["avg_words_sentences"] = df["nmb_words"] / df["nmb_sentences"]

In [14]:
df

,event,nmb_words,nmb_char,nmb_sentences,avg_char_sentences,avg_words_sentences
0,il souhaite faire une loa sur une duree de 24 ...,11,50,1,50.000000,11.000000
1,il souhaite financer le vehicule au comptant o...,12,68,2,34.000000,6.000000
2,elle est a l'ecoute des offres en loa avec un ...,13,65,3,21.666667,4.333333
3,elle ne sait pas par quelle mode de financemen...,10,54,2,27.000000,5.000000
4,il souhaite un credit pour professionnel.,6,41,2,20.500000,3.000000
...,...,...,...,...,...,...
14054,il souhaite un credit aupres de sa banque.,8,42,2,21.000000,4.000000
14055,elle fait un credit partiel aupres de sa banque.,9,48,2,24.000000,4.500000
14056,il est a l'ecoute des offres de credit partiel.,9,47,2,23.500000,4.500000
14057,il souhaite une offre pour un credit a titre c...,10,56,2,28.000000,5.000000


In [15]:
df = df.drop(['nmb_words' ,'nmb_char' ,'nmb_sentences' ,'avg_char_sentences' ,'avg_words_sentences'], axis = 1)

# Stemming Vs Lemmatization

* Connaitre le debut du mot grace  au stemming est une bonne approche statistique mais peut applicable

* Retourner à la forme canonique est difficile via Lemmatization

### Exemple du Stemming de Porter

#### EPOUSTOUFLANT
 = 
#### VCVVCCVVCCVCC
 =
#### VCVCVCVC
=
#### [VC]^3


In [16]:
# Supprimer chiffres et poncutation
#df["cleared_event"] = df["event"].str.replace("\d*[\, \! . \; \']", ' ')
df["cleared_event"] = df["event"].map(lambda x: re.sub('[,\.!?]', ' ', x)).map(lambda x: re.sub('\d', '', x))
df

,event,cleared_event
0,il souhaite faire une loa sur une duree de 24 ...,il souhaite faire une loa sur une duree de mois
1,il souhaite financer le vehicule au comptant o...,il souhaite financer le vehicule au comptant o...
2,elle est a l'ecoute des offres en loa avec un ...,elle est a l'ecoute des offres en loa avec un ...
3,elle ne sait pas par quelle mode de financemen...,elle ne sait pas par quelle mode de financemen...
4,il souhaite un credit pour professionnel.,il souhaite un credit pour professionnel
...,...,...
14054,il souhaite un credit aupres de sa banque.,il souhaite un credit aupres de sa banque
14055,elle fait un credit partiel aupres de sa banque.,elle fait un credit partiel aupres de sa banque
14056,il est a l'ecoute des offres de credit partiel.,il est a l'ecoute des offres de credit partiel
14057,il souhaite une offre pour un credit a titre c...,il souhaite une offre pour un credit a titre c...


In [17]:
from nltk.tokenize import word_tokenize
nltk.download('punkt')

[nltk_data] Downloading package punkt to /Users/greg/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [18]:
# Tokenisation
df["tokenized_event"] = df["cleared_event"].apply(lambda x: word_tokenize(str(x), language='french')) 

In [19]:
# enlever les stopwords

from nltk.corpus import stopwords
nltk.download('stopwords')

#Liste des stopwords en francais
stopWords = set(stopwords.words('french'))

df["event_wo_stopWords"] = df["tokenized_event"].apply( lambda x: ' '.join([word for word in x if word not in stopWords]))
df


[nltk_data] Downloading package stopwords to /Users/greg/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


,event,cleared_event,tokenized_event,event_wo_stopWords
0,il souhaite faire une loa sur une duree de 24 ...,il souhaite faire une loa sur une duree de mois,"[il, souhaite, faire, une, loa, sur, une, dure...",souhaite faire loa duree mois
1,il souhaite financer le vehicule au comptant o...,il souhaite financer le vehicule au comptant o...,"[il, souhaite, financer, le, vehicule, au, com...",souhaite financer vehicule comptant aupres banque
2,elle est a l'ecoute des offres en loa avec un ...,elle est a l'ecoute des offres en loa avec un ...,"[elle, est, a, l'ecoute, des, offres, en, loa,...",a l'ecoute offres loa apport eur
3,elle ne sait pas par quelle mode de financemen...,elle ne sait pas par quelle mode de financemen...,"[elle, ne, sait, pas, par, quelle, mode, de, f...",sait quelle mode financement opter
4,il souhaite un credit pour professionnel.,il souhaite un credit pour professionnel,"[il, souhaite, un, credit, pour, professionnel]",souhaite credit professionnel
...,...,...,...,...
14054,il souhaite un credit aupres de sa banque.,il souhaite un credit aupres de sa banque,"[il, souhaite, un, credit, aupres, de, sa, ban...",souhaite credit aupres banque
14055,elle fait un credit partiel aupres de sa banque.,elle fait un credit partiel aupres de sa banque,"[elle, fait, un, credit, partiel, aupres, de, ...",fait credit partiel aupres banque
14056,il est a l'ecoute des offres de credit partiel.,il est a l'ecoute des offres de credit partiel,"[il, est, a, l'ecoute, des, offres, de, credit...",a l'ecoute offres credit partiel
14057,il souhaite une offre pour un credit a titre c...,il souhaite une offre pour un credit a titre c...,"[il, souhaite, une, offre, pour, un, credit, a...",souhaite offre credit a titre comparatif


In [20]:
# Stimmer
from nltk.stem.snowball import SnowballStemmer

ps = SnowballStemmer(language='french')
df["stemmed_event"] = df["event_wo_stopWords"].apply(lambda x: ps.stem(x))
df[['event_wo_stopWords',	'stemmed_event']].head(20)


,event_wo_stopWords,stemmed_event
0,souhaite faire loa duree mois,souhaite faire loa duree mois
1,souhaite financer vehicule comptant aupres banque,souhaite financer vehicule comptant aupres banqu
2,a l'ecoute offres loa apport eur,a l'ecoute offres loa apport eur
3,sait quelle mode financement opter,sait quelle mode financement opt
4,souhaite credit professionnel,souhaite credit professionnel
5,ouvert a plus informations credit,ouvert a plus informations cred
6,souhaite faire credit connaitre modalites,souhaite faire credit connaitre modalit
7,fait credit banque,fait credit banqu
8,interesse,interess
9,fait credit banque,fait credit banqu


In [21]:
#Lemmatisation (en utilisant les stopwords comme reference)
import spacy
from spacy_lefff import LefffLemmatizer
from spacy.language import Language

In [29]:
nlp = spacy.load('fr_core_news_sm')
doc = nlp("".join(df["event_wo_stopWords"]))
for token in doc:
	print(token, token.lemma_)


souhaite souhaite
faire faire
loa loa
duree dure
moissouhaite moissouhaite
financer financer
vehicule vehicule
comptant compter
aupres aupre
banquea banquea
l' le
ecoute ecoute
offres offre
loa loa
apport apport
eursait eurser
quelle quel
mode mode
financement financement
optersouhaite optersouhait
credit credit
professionnelouvert professionnelouvert
a avoir
plus plus
informations information
creditsouhaite creditsouhait
faire faire
credit credit
connaitre connaitr
modalitesfait modalitesfait
credit credit
banqueinteressefait banqueinteressefer
credit credit
banqueouvert banqueouvert
financementsdesire financementsdesir
offre offrir
financement financement
societeinteressee societeinteresse
offres offre
creditn' creditn'
est être
interesseeinteressee interesseeinteresse
offres offre
locationinteresse locationinteress
offres offre
credit credit
apport apport
interesse interess
extensions extension
garantieinteressee garantieinteresse
offres offre
apport apport
eur eur
moisdesire moisde

In [28]:
#df[["event_wo_stopWords","lemmatized_event"]]
df["lemmized"]

0        4378518358114130125
1        4378518358114130125
2        4378518358114130125
3        4378518358114130125
4        4378518358114130125
                ...         
14054    4378518358114130125
14055    4378518358114130125
14056    4378518358114130125
14057    4378518358114130125
14058    4378518358114130125
Name: lemmized, Length: 14059, dtype: int64

In [24]:
#WordCloud
wordcloud = WordCloud(max_font_size=30, max_words=10, background_color="black").generate(str(df["event_wo_stopWords"]))
plt.figure(figsize=(10, 20))
plt.imshow(wordcloud, interpolation="bilinear")
plt.axis("off")
plt.show()

TypeError: 'module' object is not callable

In [ ]:
from sklearn.feature_extraction.text import TfidfTransformer
# On modifie juste pour ajouter une branche
